# Usage Examples and General Considerations

This notebook explores different situations in Difference Image Analysis (DIA) using the methods provided by `ois`.


In [ ]:
import ois
import numpy as np
import sep
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(seed=12)

In [ ]:
ois.__version__

In [ ]:
def plot_images(img_g, ref_g):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img_g, cmap='gray', interpolation='none', origin='lower')
    axes[0].axis('off')
    axes[0].set_title("Science Image")

    axes[1].imshow(ref_g, cmap='gray', interpolation='none', origin='lower')
    axes[1].axis('off')
    axes[1].set_title("Reference Image")

    axes[2].imshow(img_g - ref_g, cmap='gray', interpolation='none', origin='lower')
    axes[2].axis('off')
    axes[2].set_title("Plain Difference")

    plt.tight_layout()
    plt.show()
    
    
def plot_diffs(all_diffs, all_krns, method_names):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    for diff, m_name, ax in zip(all_diffs, method_names, axes[0]):
        ax.imshow(diff, cmap='gray', interpolation='none', origin='lower')
        ax.axis('off')
        ax.set_title(m_name)

    for krn, m_name, ax in zip(all_krns, method_names, axes[1]):
        ax.imshow(krn, interpolation='none', origin='lower')
        ax.axis('off')
        ax.set_title(m_name)

    plt.tight_layout()
    plt.show()

-----------------------
# Case 1:

### Two identical images with Gaussian PSF's

In the case where we have two identical (or very similar) images, it can be verified that:

* A naive Alard-Lupton with an incorrectly tuned Gaussian can't reproduce the identity kernel and the subtraction will have characteristic rings around sources.
* A correctly tuned Alard-Lupton performs very well.
* Bramich (and AdaptiveBramich) can pick the correct kernel without tuning extra parameters other than the kernel side.



In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f

ref = img.copy()

# Let's add the same Gaussian PSF response to both images
from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.5, mode='constant')
ref_g = gaussian_filter(ref, sigma=1.5, mode='constant')


# Plot the images
plot_images(img_g, ref_g)

In [ ]:
method_names = ["Alard-Lupton", "Alard-Lupton", "Bramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy':1.0, 'modPolyDeg':0}]},
       {'gausslist': [{'sx': 0.01, 'sy':0.01, 'modPolyDeg':0}]},
       {}]
krn_shape = (5, 5)
# We store all the differences and kernels in the following lists
all_diffs = []
all_krns = []

# Do for the all the methods
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name, bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)


# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
# Errors:
for name, diff in zip(method_names, all_diffs):
    print("{} L1 error: {:.2g}".format(name, np.abs(diff).max()))

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-3)
    print("{} sources found in method {}.".format(len(sources), mname))

---------------------------
# Case 2:

### Bramich can fix small translations

A naive Alard-Lupton won't fix small translations unless we introduce it explicitely in the center of the Gaussian kernel.

Bramich (and AdaptiveBramich) will solve small translations automatically.


In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f
    
ref = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    ref[y + 1, x + 2] = f  # We add a small translation here

# Let's add a Gaussian PSF response with same seeing for both images
from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.5, mode='constant')
ref_g = gaussian_filter(ref, sigma=1.5, mode='constant')

# Plot the images
plot_images(img_g, ref_g)

In [ ]:
method_names = ["Alard-Lupton", "Alard-Lupton", "Bramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy':1.0, 'modPolyDeg':0}]},
       {'gausslist': [{'sx': 0.01, 'sy':0.01, 'center': (1, 0), 'modPolyDeg':0}]},
       {}]

krn_shape = (5, 5)
all_diffs = []
all_krns = []
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name,
        bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)

# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
# Errors:
for name, diff in zip(method_names, all_diffs):
    print("{} L1 error: {:.2g}".format(name, np.abs(diff).max()))

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-3)
    print("{} sources found in method {}.".format(len(sources), mname))

-------------

# Case 3

### Different seeing

The most common case is when we have images with different _seeing_.

Assuming

$R \approx I \otimes K$

* The ideal convolution kernel for an image that have a Gaussian seeing PSF $\sigma_{img}$ and a reference with     $\sigma_{ref}$ is

$\sigma_K = \sqrt{\sigma_{img}^2 - \sigma_{ref}^2}$

(Assuming $ \sigma_{img} > \sigma_{ref} $)

* If Alard-Lupton is not tuned for the correct Gaussian std-dev, it will leave behind rings at the sources positions.
* The delta basis will pick the correct convolution kernel.
* Make sure the dimensions of the kernel are enough to contain all the features of the expected kernel.


In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f
    
ref = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    ref[y, x] = f

# Let's add a Gaussian PSF response with same seeing for both images
from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.7, mode='constant')
ref_g = gaussian_filter(ref, sigma=0.8, mode='constant')

# Plot the images
plot_images(img_g, ref_g)

In [ ]:
s_tuned = np.sqrt(1.7 ** 2 - 0.8 ** 2)
s_tuned

In [ ]:
method_names = ["Alard-Lupton", "Alard-Lupton", "Bramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy': 1.0, 'modPolyDeg':0}]},
       {'gausslist': [{'sx': s_tuned, 'sy':s_tuned, 'modPolyDeg':0}]},
       {}]

krn_shape = (11, 11)
all_diffs = []
all_krns = []
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name,
        bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)
    
# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
# Errors:
for name, diff in zip(method_names, all_diffs):
    print("{} L1 error: {:.2g}".format(name, np.abs(diff).max()))

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-3)
    print("{} sources found in method {}.".format(len(sources), mname))
    
print("\nIf we raise the tolerance to 1E-2:")
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-2)
    print("{} sources found in method {}.".format(len(sources), mname))

--------------------
# Case 4

### In the presence of a transient

* The methods are unaffected with the presence of a single transient.
* None of the methods distort the transient noticeably.



In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f

ref = img.copy()

transient_pos = (120, 17)
transient_flux = 350
x, y = transient_pos
img[y, x] = transient_flux

# Let's add a Gaussian PSF response with different seeing for both images
from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.5, mode='constant')
ref_g = gaussian_filter(ref, sigma=1.5, mode='constant')

# Plot the images
plot_images(img_g, ref_g)

In [ ]:
method_names = ["Alard-Lupton", "Alard-Lupton", "Bramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy':1.0, 'modPolyDeg':0}]},
       {'gausslist': [{'sx': 0.01, 'sy':0.01, 'modPolyDeg':0}]},
       {}]
krn_shape = (5, 5)
# We store all the differences and kernels in the following lists
all_diffs = []
all_krns = []

# Do for the all the methods
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name, bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)

# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-3)
    print("{} sources found in method {}.".format(len(sources), mname))

----------------------
# Case 5:

### AdaptiveBramich can fix small rotations

Using a quadratic variation across the image field, Adaptive Bramich can fix small rotations.

In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f

from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.5, mode='constant')

from scipy.ndimage import rotate

ref = rotate(img_g, 0.05)
hr, wr = ref.shape
ref_g = ref[(hr - h) // 2 + (hr - h) % 2: -(hr - h) // 2 or None,
            (wr - w) // 2 + (wr - w) % 2: -(wr - w) // 2 or None]

# Plot the images
plot_images(img_g, ref_g)

In [ ]:
method_names = ["Alard-Lupton", "Bramich", "AdaptiveBramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy':1.0, 'modPolyDeg':0}]},
       {}, {'poly_degree': 2}]
krn_shape = (5, 5)
# We store all the differences and kernels in the following lists
all_diffs = []
all_krns = []

# Do for the all the methods
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name, bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)
all_krns[2] = np.zeros(krn_shape)  # An adaptive kernel is a 4D object, so it can't be graphed

# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
# Errors:
for name, diff in zip(method_names, all_diffs):
    print("{} L1 error: {:.2g}".format(name, np.abs(diff).max()))

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff, thresh=1E-2)
    print("{} sources found in method {}.".format(len(sources), mname))

--------------

# Case 6:

### Adding even a little bit of noise can spoil the subtraction considerably

This is like Case 1 but with noise added.

* Alard-Lupton seems to be more robust against added noise.

In [ ]:
h, w = img_shape = (200, 200)
n_stars = 10
pos_x = np.random.randint(10, w - 10, n_stars)
pos_y = np.random.randint(10, h - 10, n_stars)
fluxes = 200.0 + np.random.rand(n_stars) * 300.0

img = np.zeros(img_shape)
for x, y, f in zip(pos_x, pos_y, fluxes):
    img[y, x] = f

ref = img.copy()

# Let's add a Gaussian PSF response with different seeing for both images
from scipy.ndimage.filters import gaussian_filter
img_g = gaussian_filter(img, sigma=1.5, mode='constant')
ref_g = gaussian_filter(ref, sigma=1.5, mode='constant')

# Let's add some white noise:
noise_dc = 5.0
noise_std = np.sqrt(noise_dc)
img_g += np.random.normal(loc=noise_dc, scale=noise_std, size=img.shape)
ref_g += np.random.normal(loc=noise_dc, scale=noise_std, size=ref.shape)

# Plot the images
plot_images(img_g, ref_g)

In [ ]:
method_names = ["Alard-Lupton", "Alard-Lupton", "Bramich"]
kws = [{'gausslist': [{'sx': 1.0, 'sy':1.0, 'modPolyDeg':0}]},
       {'gausslist': [{'sx': 0.01, 'sy':0.01, 'modPolyDeg':0}]},
       {}]
krn_shape = (5, 5)
# We store all the differences and kernels in the following lists
all_diffs = []
all_krns = []

# Do for the all the methods
for kw, m_name in zip(kws, method_names):
    %time
    diff, __, krn, __ = ois.optimal_system(
        img_g, ref_g, kernelshape=krn_shape,
        method=m_name, bkgdegree=None,
        **kw,
    )
    all_diffs.append(diff)
    all_krns.append(krn)
    
# Plot the subtractions
plot_diffs(all_diffs, all_krns, method_names)

In [ ]:
# Errors:
for name, diff in zip(method_names, all_diffs):
    print("{} L1 error: {:.2g}".format(name, np.abs(diff).max()))

In [ ]:
for diff, mname in zip(all_diffs, method_names):
    bkg = sep.Background(diff)
    sources = sep.extract(diff - bkg.back(), thresh=2.0 * bkg.globalrms)
    print("{} sources found in method {}.".format(len(sources), mname))